# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [100]:
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv
from prettytable import PrettyTable

#### Creating list of filepaths to process original event csv data files

In [2]:
filepath = os.getcwd() + '/event_data'

for root, dirs, files in os.walk(filepath):
    
    file_path_list = glob.glob(os.path.join(root,'*'))
    print(file_path_list)

['/home/workspace/event_data/2018-11-30-events.csv', '/home/workspace/event_data/2018-11-23-events.csv', '/home/workspace/event_data/2018-11-22-events.csv', '/home/workspace/event_data/2018-11-29-events.csv', '/home/workspace/event_data/2018-11-11-events.csv', '/home/workspace/event_data/2018-11-14-events.csv', '/home/workspace/event_data/2018-11-20-events.csv', '/home/workspace/event_data/2018-11-15-events.csv', '/home/workspace/event_data/2018-11-05-events.csv', '/home/workspace/event_data/2018-11-28-events.csv', '/home/workspace/event_data/2018-11-25-events.csv', '/home/workspace/event_data/2018-11-16-events.csv', '/home/workspace/event_data/2018-11-18-events.csv', '/home/workspace/event_data/2018-11-24-events.csv', '/home/workspace/event_data/2018-11-04-events.csv', '/home/workspace/event_data/2018-11-19-events.csv', '/home/workspace/event_data/2018-11-26-events.csv', '/home/workspace/event_data/2018-11-12-events.csv', '/home/workspace/event_data/2018-11-27-events.csv', '/home/work

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [69]:
full_data_rows_list = [] 
    
for f in file_path_list:

    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            

csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))
        print(row[2] + "," + row[13])

Celeste,Playa Haters
Celeste,You Belong With Me
Celeste,Valerie
Celeste,Dizzy
Anabelle,Supelicula
Connar,Bermuda
Jayden,Din Din Wo
Sara,Eddie's Gun
Sara,Hook
Sara,Shiver
Sara,The Wild One_ Forever
Sara,Give and Go
Sara,Cosmic Love
Sara,Greece 2000
Sara,Sorry
Sara,Round And Round (Soul Mix Edit)
Sara,Windmills Of Your Mind
Sara,Tequila
Sara,Poor Leno Jakatta Radio Mix
Sara,Bat Country (Album Version)
Sara,Ain't Misbehavin
Sara,Hallaa
Sara,Cathar Rhythm
Sylvie,Moonlight Serenade
Sara,KissKiss
Lily,Rabbit Heart (Raise It Up)
Sara,Me_ Myself & I
Sara,Got Money
Lily,Milord
Lily,Losing Ground
Sara,Elevator
Lily,I Need You Now (Build A Bridge Version)
Sara,Whataya Want From Me
Sara,Little Girl (With Blue Eyes)
Lily,Creil City
Sara,Ponderosa
Lily,Betterman (full-length/album version)
Sara,Believe (Demo)
Lily,Fader
Sara,Patadas En El Corazon
Lily,Finger Twist & Split (Album Version)
Sara,Bubble Toes
Lily,Simple
Sara,Feelin' Blue
Lily,No Seas Cruel (vivo)
Sara,Not The Sun
Lily,Into The Sun
Sara,

In [17]:
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

267


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
from cassandra.cluster import Cluster

try:
    #cluster = Cluster(['127.0.0.1']) #Local
    cluster = Cluster()
    session = cluster.connect()
    print('connected success')
except Exception as e:
    print(e)

connected success


#### Create Keyspace

In [6]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS public
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor': 1 }"""
                   )
    print('keyspace created')
except Exception as e:
    print(e)

keyspace created


#### Set Keyspace

In [7]:
try:
    session.set_keyspace('public')
    print('keyspace set')
except Exception as e:
    print(e)

keyspace set


### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Query 1:  Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession = 4

In [131]:
def insert_data(query, session, values, dtypes, file):
    assert(len(values) ==  len(dtypes))
    with open(file, encoding = 'utf8') as f:
        csvreader = csv.reader(f)
        next(csvreader) # skip header
        for line in csvreader:
            session.execute(query, tuple([dtype(line[x]) for x, dtype in zip(values, dtypes)]))

In [132]:
query = "CREATE TABLE IF NOT EXISTS artist_by_item "
query = query + "(sessionId int, itemInSession int, artist text, song text, length decimal, PRIMARY KEY ((sessionId), itemInSession) );"

try:
    session.execute(query)
except Exception as e:
    print(e)

In [133]:
file = 'event_datafile_new.csv'

query = "INSERT INTO artist_by_item (sessionId, itemInSession, artist, song, length)"
query = query + " VALUES (%s, %s, %s, %s, %s)"

values = [8, 3, 0, 9, 5]
dtypes = [int, int, str, str, float]

insert_data(query, session, values, dtypes, file)

#### Do a SELECT to verify that the data have been inserted into each table

In [134]:
query = "Select artist, song, length from artist_by_item where sessionId = 139 and itemInSession = 1; "

q1_table = PrettyTable()
q1_table.field_names = ['Artist', 'Song', 'Length']


try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    q1_table.add_row([row.artist, row.song, row.length])

print(q1_table)

+---------+--------------+-----------+
|  Artist |     Song     |   Length  |
+---------+--------------+-----------+
| Des'ree | You Gotta Be | 246.30812 |
+---------+--------------+-----------+


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

## Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)  for userid = 10, sessionid = 182

In [178]:
query = "CREATE TABLE IF NOT EXISTS artist_by_userid_sessionid "
query = query + "(userId int, sessionId int, itemInSession int, artist text, song text, firstName text, lastName text,  PRIMARY KEY ((userid, sessionid), itemInSession) );"

try:
    session.execute(query)
except Exception as e:
    print(e)                   

In [179]:
file = 'event_datafile_new.csv'

query = "INSERT INTO artist_by_userid_sessionid (userId, sessionId, itemInSession, artist, song, firstName, lastName)"
query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"

values = [10, 8, 3, 0, 9, 1, 4]
dtypes = [int, int, int, str, str, str, str]

insert_data(query, session, values, dtypes, file)

In [182]:
query = "Select artist, song, firstname, lastname from artist_by_userid_sessionid where userid = 10 and sessionid = 182; "

q2_table = PrettyTable()
q2_table.field_names = ['Artist', 'Song', 'Username']

try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    q2_table.add_row([row.artist, row.song, row.firstname + " " + row.lastname])
    
print(q2_table)

+-------------------+------------------------------------------------------+-------------+
|       Artist      |                         Song                         |   Username  |
+-------------------+------------------------------------------------------+-------------+
|  Down To The Bone |                  Keep On Keepin' On                  | Sylvie Cruz |
|    Three Drives   |                     Greece 2000                      | Sylvie Cruz |
| Sebastien Tellier |                      Kilometer                       | Sylvie Cruz |
|   Lonnie Gordon   | Catch You Baby (Steve Pitron & Max Sanna Radio Edit) | Sylvie Cruz |
+-------------------+------------------------------------------------------+-------------+


## Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'             

In [190]:
query = "CREATE TABLE IF NOT EXISTS username_by_song "
query = query + "(song text, firstname text, lastname text,  PRIMARY KEY ((song), firstname, lastname) );"

try:
    session.execute(query)
except Exception as e:
    print(e)    

In [191]:
file = 'event_datafile_new.csv'

query = "INSERT INTO username_by_song (song, firstname, lastname) "
query = query + " VALUES (%s, %s, %s)"

values = [9, 1, 4]
dtypes = [str, str, str]

insert_data(query, session, values, dtypes, file)

In [196]:
query = "Select firstname, lastname from username_by_song where song = 'Greece 2000'; "

q3_table = PrettyTable()
q3_table.field_names = ['Username']

try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    q3_table.add_row([row.firstname + " " + row.lastname])
print(q3_table)

+--------------+
|   Username   |
+--------------+
| Sara Johnson |
| Sylvie Cruz  |
+--------------+


### Drop the tables before closing out the sessions

In [189]:
try:
    session.execute("drop table IF EXISTS artist_by_item; ")
    session.execute("drop table IF EXISTS artist_by_userid_sessionid; ")
    session.execute("drop table IF EXISTS username_by_song; ")
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [ ]:
session.shutdown()
cluster.shutdown()